In [1]:
import joblib
import re
import string
from nltk.stem import PorterStemmer
import json
import nltk

In [6]:
# Text preprocessing
def wordopt(text):
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

ps = PorterStemmer()
with open('static/detector/models/stopwords-tl.json', 'r') as f:
    stopwords = json.load(f)

def preprocess(text):
    text = wordopt(text)
    tokens = nltk.word_tokenize(text)
    stems = [ps.stem(token) for token in tokens]
    filtered = [stem for stem in stems if stem not in stopwords]
    return filtered

def predict(text):
    processed = preprocess(text)

    # Load vectorizer
    vectorizer = joblib.load('static/detector/models/vectorizer_mlp.joblib')

    # Transform the validation data using the same vectorizer
    tfidf = vectorizer.transform(processed)

    model = joblib.load('static/detector/models/stack_mlp.joblib')  # Update with your model path
    # Make prediction using the loaded model
    # prediction = model.predict_proba(tfidf)[0]  # Assuming binary prediction (0 or 1)
    prediction = model.predict_proba(tfidf)[0]  # Assuming binary prediction (0 or 1)
    value1 = prediction[0]  # Probability for class 0
    value2 = prediction[1]  # Probability for class 1
    return prediction

In [8]:
input_text = """Nitong Huwebes, Abril 11, inilabas ng Pulse Asia Research, Inc. ang resulta ng latest nilang survey para sa senatorial election. Pawang mga reelectionists ang top 3 sa kanilang listahan. Ayon sa resulta ng survey na isinagawa noong Marso 23 hanggang Marso 27, 2019 ay nangunguna sa listahan si Grace Poe na nakakuha ng 72. 6 percent voter preference. Sunod naman si Cynthia Villar na may 63.7 percent at pangatlo si Sonny Angara na may 58.5 percent. Tatlo naman sa mga pambato ni Pangulong Rodrigo Duterte ang nakapasok sa Top 12: sina Bong Go, Bato Dela Rosa, at Francis Tolentino na tumatakbo sa ilalim ng PDP-Laban. Nasa 1,800 katao na may gulang 18 pataas ang respondents sa survey. Kung pagbabatayan ang mga komento sa Pulse Asia Facebook post, marami ang natutuwa sa pagiging number 1 ni Poe. Ayon sa komento ng Facebook user na si L Chu, "Sobrang deserve mo yan, karapat-dapat ka talaga sa pwestong yan dahil sa 'yong sipag at tiyaga na maglingkod sa mamamayan." Tinutukoy niya si Poe. "Consistent pa rin talaga si Senator Poe! No doubt! She will be the no. 1 senator after election!," komento naman ni D Cruz. "Be 'Grace Poe' in this world full of inconsistencies," sabi ni C Lovero. May mga negatibong komento naman hinggil sa pagiging no.1 ni Poe. "Kunwari totoong survey...para lang ipasok si Grace Poe!!!" komento ni A Bulls. "Conflict of interest si PoeLse este Poe kasi correct me if I'm wrong kamag-anak niya may-ari o presidente ng Pulse Asia..." komento ni M Lamponi. May gumamit din ng #TraPoe. Gayunman, may mga tumutul1gsa rin mismo sa survey at sinabing ito ay mind conditioning lamang. May kumuwestiyon din kung bakit walang Facebook survey ang Pulse Asia. May nagrekomenda pa nga na gawin ang survey sa Facebook. "Sa FB kayo magsurvey para makasali naman kami dito sa HK at iba pang lugar sa labas ng Pinas. (Para) medyo makatotohanan ang resulta, 'di ba?" komento ni R Rick. Samantala, sa isang press release ay nagpasalamat si Poe sa mga patuloy na sumusuporta sa kaniya. "Taos-puso ang ating pasasalamat sa patuloy na tiwala ng ating mga kababayan. Ang kampanyang ito ay para sa kanila," pahayag ni Poe.
"""
# input_text = 'Humihiling ngayon ng tulong ang 12-years old brain cancer patient na si Jinn Sam sa mga taong may mabubuting puso at'
print(predict(input_text))

[0.99544205 0.00455795]


In [ ]:
input_text = 'Sapu kamuwang-muwang at ang napansin nga namin ang malakas na usok ay napupunta sa bata, makikita nantin sa surveilance iyan. Ibig sabihin ay talamak na talaga itong mag-asawan kontra-droga? Source: GMA News'
# input_text = 'Humihiling ngayon ng tulong ang 12-years old brain cancer patient na si Jinn Sam sa mga taong may mabubuting puso at'
value1, value2 = predict(input_text)


# Now you can use value1 and value2 separately
print("Probability for class 0:", value1)
print("Probability for class 1:", value2)
# Class 0 = False
# Class 1 = True

In [4]:
# input_text = 'Sapu kamuwang-muwang at ang napansin nga namin ang malakas na usok ay napupunta sa bata, makikita nantin sa surveilance iyan. Ibig sabihin ay talamak na talaga itong mag-asawan kontra-droga? Source: GMA News'
input_text2 = 'Humihiling ngayon ng tulong ang 12-years old brain cancer patient na si Jinn Sam sa mga taong may mabubuting puso at'
value1, value2 = predict(input_text2)


# Now you can use value1 and value2 separately
print("Probability for class 0:", value1)
print("Probability for class 1:", value2)
# Class 0 = False
# Class 1 = True

Probability for class 0: 0.9925405159401599
Probability for class 1: 0.007459484059840081


Instead of using a threshold, we will be using Weighted Decision.

If value1 == value2 (which means 50:50), it will interpret as Fake News.

Why?
Class 0 (Fake Classe) is more important than the other. In the application, false negatives are more critical. A false negative occurs when the model fails to identify a piece of news as fake when it is indeed fake. This means the fake news goes undetected or is mistakenly classified as genuine.
Reducing false negatives is crucial in fake news detection to ensure the effectiveness of the detection system